In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aws-s3/s3-userguide.pdf
/kaggle/input/aws-s3/glacier-dg.pdf
/kaggle/input/aws-s3/s3-outposts.pdf
/kaggle/input/aws-s3/s3-api.pdf


In [2]:
%pip install PyPDF2
%pip install langchain
%pip install langchain_community
%pip install langchain_google_genai
%pip install langchain_text_splitters
%pip install sentence-transformers
%pip install faiss-cpu
%pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.7/343.7 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.72
    Uninstalling langchain-core-0.3.72:
      Successfully uninstalled langchain-core-0.3.72
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.9
    Uninstalling langchain-text-splitters-0.3.9:
      Successfully uninstalled langchain-text-splitters-0.3.9
ERROR: pip's dependency resolver does not currently ta

In [4]:
from PyPDF2 import PdfReader
from langchain_community.llms import Cohere
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts  import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

2026-01-05 09:53:31.398546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767606811.620074      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767606811.687560      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:
pdf_file1 = open('/kaggle/input/aws-s3/glacier-dg.pdf','rb')
pdf_file2 = open('/kaggle/input/aws-s3/s3-api.pdf','rb')
pdf_file3 = open('/kaggle/input/aws-s3/s3-outposts.pdf','rb')
pdf_file4 = open('/kaggle/input/aws-s3/s3-userguide.pdf','rb')

In [8]:
pdf_text1 = ""
pdf_reader = PdfReader(pdf_file1)
for page in pdf_reader.pages:
    pdf_text1 += page.extract_text()

pdf_text2 = ""
pdf_reader = PdfReader(pdf_file2)
for page in pdf_reader.pages:
    pdf_text2 += page.extract_text()

pdf_text3 = ""
pdf_reader = PdfReader(pdf_file3)
for page in pdf_reader.pages:
    pdf_text3 += page.extract_text()

pdf_text4 = ""
pdf_reader = PdfReader(pdf_file4)
for page in pdf_reader.pages:
    pdf_text4 += page.extract_text()

In [9]:
all_text = pdf_text1 + '\n' + pdf_text2 + '\n' + pdf_text3 + '\n' + pdf_text4
len(all_text)

11685358